# Yahtzee AI

## Rules of Yahtzee

Roll 5 dice. Any number of them can be rerolled twice, so 3 times total. Then fill one box corresponding to table below, either with the respective points, or with 0 if one doesn't want to or is not able to do otherwise.

| Shape                 | What to score        | Points |
| --------------------- | -------------------- | ------ |
| Ones                  | Total of Ones only   |        |
| Twos                  | Total of Twos only   |        |
| Threes                | Total of Threes only |        |
| Fours                 | Total of Fours only  |        |
| Fives                 | Total of Fives only  |        |
| Sixes                 | Total of Sixes only  |        |
| If 63+ points above   | 35 point bonus       |        |
| 3 of a Kind           | Total of all 5 dice  |        |
| 4 of a Kind           | Total of all 5 dice  |        |
| Full House            | 25 points            |        |
| Small Straight        | 30 points            |        |
| Large Straight        | 40 points            |        |
| Yahtzee (5 of a kind) | 50 points            |        |
| Chance                | Total of all 5 dice  |        |

If a minimum of 63 points are achieved in upper section, a 35 point bonus is issued.

If one has already rolled a Yahtzee and does so again, they get a 100-point bonus and must score the total of all 5 dice in the correcsponding box in the upper section. If that is already filled in, they can score in any open lower section box with the respective counting rules. If those are all filled, they must enter a zero somewhere in the upper section.

If one roles a Yahtzee, but has already entered zero in the Yahtzee box, they do not get a bonus, but can still enter it according to the order described above.

The game ends when all 13 boxes are filled, at which point the score is summed up, the more the better.

Detailed rules can be found under:
https://www.hasbro.com/common/instruct/Yahtzee.pdf

# Thoughts on complexity

## Number of possible game states:

Ones-Sixes: Empty, Zero, 5 additional score options

3/4 of a kind: Empty, Zero, 26 additional score options

Full House, Small/Large Straight, Yahtze: Empty, Zero, 1 score option

Chance: Empty, Zero, 26 score options

Plus 6 possible values for occurences of all 6 numbers in dice role.

And 3 options for rolls left (2, 1, none)

All in all $7^6 \cdot 28^2 \cdot 3^4 \cdot 28 \cdot 6^6 \cdot 3 = ~3x10^{16}$ possible states

In [69]:
import numpy as np
from numpy.random import default_rng

In [70]:
# Rolls a specified number of dice and returns them as an array
def dice_roll(number):
    # set up random number generator
    rng = default_rng()
    roll = rng.integers(1, 7, size=5)
    return roll

ERROR! Session/line number was not unique in database. History logging moved to new session 280


In [71]:
## Score Card ##
# Index 0 # Ones #
# Index 1 # Twos #
# Index 2 # Threes #
# Index 3 # Fours #
# Index 4 # Fives #
# Index 5 # Sixes #
# Index 6 # 3 of a Kind #
# Index 7 # 4 of a Kind #
# Index 8 # Full House #
# Index 9 # Small Straight #
# Index 10 # large Straight #
# Index 11 # Yahtzee #
# Index 12 # Chance #

# Initializes score card with -1, representing empty
def init_score_card():
    return np.full(13, -1)

In [72]:
# Checks for the validity of special throws one can make
# The functions give back boolean whether this throw is possible for the dice entered

def three_of_a_kind(dice):
    for i in range(3):
        if (dice==dice[i]).sum() >= 3:
            return True
    return False
    
def four_of_a_kind(dice):
    for i in range(2):
        if (dice==dice[i]).sum() >= 4:
            return True
    return False
    
def full_house(dice):
    first_number = dice[0]
    first_sum = (dice==first_number).sum()
    if first_sum < 2:
        return False
    for i in range(4):
        if dice[i+1] != first_number:
            second_number = dice[i+1]
            second_sum = (dice==second_number).sum()
            if second_sum == 2 and first_sum == 3:
                return True
            elif second_sum == 3 and first_sum == 2:
                return True
            else:
                return False
    return False
    
def small_straight(dice):
    sorted_dice = np.unique(dice)
    counter = 0
    dice_len = len(sorted_dice)
    if (dice_len < 4):
        return False
    for i in range(dice_len-1):
        if sorted_dice[i]+1==sorted_dice[i+1]:
            counter += 1
        else:
            counter = 0
    if counter >= 3:
        return True
    else:
        return False
    
def large_straight(dice):
    sorted_dice = np.unique(dice)
    if len(sorted_dice) < 5:
        return False
    for i in range(4):
        if sorted_dice[i]+1!=sorted_dice[i+1]:
            return False
    return True

# TODO: Implement bonus rule
def yahtzee(dice):
    if (dice==dice[0]).sum() == 5:
        return True
    else:
        return False

In [73]:
def calculate_score(score_card):
    """
        Calculates the final score of a finished game.
        
        Args:
            score_card (array): The final game state that should be scored
        
        Returns:
            The final score achieved
    """
    score = sum(score_card)
    if sum(score_card[:6])>=63:
        score += 35
    return score

In [74]:
# Enter move as number between 0 and 12
# Computes and returns new score card state and score given
def make_move(state, move, dice):
    score = 0
    new_state = state.copy()
    if move < 6:
        score = count_dice(move, dice)
    elif move == 6 and three_of_a_kind(dice):
        score = count_dice(-1, dice)
    elif move == 7 and four_of_a_kind(dice):
        score = count_dice(-1, dice)
    elif move == 8 and full_house(dice):
        score = 25
    elif move == 9 and small_straight(dice):
        score = 30
    elif move == 10 and large_straight(dice):
        score = 40
    elif move == 11 and yahtzee(dice):
        # here special rules need to be added
        score = 50
    elif move == 12:
        score = count_dice(-1, dice)
    new_state[move] = score

    return new_state, score

In [75]:
# Counts the score for the different throws
# If move=-1, all dice are counted, otherwise just the ones of the respective number
def count_dice(move, dice):
    if move == -1:
        return np.sum(dice)
    else:
        return (dice == move+1).sum()*(move+1)

# Thoughts on agent decision making

### Value-Network

tries to estimate the expected score for a given board state
Then when deciding on a move, let the value-network evaluate the positions resulting from each legal move (which are known) and simply pick the highest rated. (for training with a certain probability, so that exploration is possible)
For training do that until game is finished and then feed the results of the different moves back to the respective network to train.

### How to handle deciding which dice to reroll?

Maybe some sort of random markov process? And make decision where possible immediate score is highest?
But that doesn't account for long term strategies.
Could maybe also done by some statistical calculation? But that has same problem
Maybe delay problem and first do Yahtzee with just one dice roll?

In [140]:
def evaluate(game_state):
    """
    Should somehow evaluate how good a game state is.
    
    Args:
   game_state (): The game state that should be evaluated
        
    Returns:
        An evaluation of the game_state between
    """
    # random strategy
    # evaluation = 1
    
    # greedy strategy
    temp = game_state.copy()
    temp[temp==-1] = 0
    evaluation = calculate_score(temp)
    
    return evaluation

def choose_move(score_card, dice):
    """
    Decides which move to make. Probabilistically choose depending on evaluatiion of resulting state.
    
    Args:
        score_card (array): The current state of the game
    Returns:
        The index of the move that is chosen
    """
    possible_moves = np.where(score_card==-1)[0]
    scores = [evaluate(make_move(score_card, move, dice)[0]) for move in possible_moves]
    scores_norm = scores / np.sum(scores)
    rng = default_rng()
    
    # plays move with probability proportional to evaluation of resulting states
    #return rng.choice(a=possible_moves, p=scores_norm)
    # plays the move that produces maximum evaluation
    return possible_moves[np.argmax(scores)]
        

In [107]:
def play_round():
    """
    Plays one round of Yahtzee using.
    
    Returns:
        The final score card and the final score that is produced
    """
    score_card = init_score_card()

    # random moves
    for i in range(13):
        dice = dice_roll(5)
        move = choose_move(score_card, dice)
        score_card, score = make_move(score_card, move, dice)
    final_score = calculate_score(score_card)
    return score_card, final_score

In [148]:
# Play 1000 rounds of Yahtzee with a and return the average and max score
# number of rounds played
n = 10000
data = np.empty(n)
example_card = None
for i in range(n):
    data[i] = play_round()[1]
print('Mean: ', np.mean(data))
print('Max: ', np.max(data))

Mean:  109.2008
Max:  220.0


# Scores

### Random

With n=10000, Mean=45.55, Max=140

### Semi-greedy

Evaluation just gives back score if summed up at that point

With n=10000, Mean=57.44, Max=155

### Greedy

Always chooses the max

With n=10000, Mean=109.2, Max=220